## Train Yolo on court keypoints

### Download dataset

In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="KZo2u7XZFUW0de237okd")
project = rf.workspace("fyp-3bwmg").project("reloc2-den7l")
version = project.version(1)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to reloc2-1 in yolov8:: 100%|██████████| 2948/2948 [00:03<00:00, 786.93it/s] 


### Setup: Download Pre-trained Model
(downloads yolov8x-pose to desired path: models/pretrained/)

In [1]:
from pathlib import Path
from ultralytics import YOLO

# Ensure models directory exists
models_dir = Path('../models/pretrained')
models_dir.mkdir(parents=True, exist_ok=True)

# Download pretrained model if not exists
model_path = models_dir / 'yolov8x-pose.pt'
if not model_path.exists():
    print(f"Downloading yolov8x-pose.pt to {model_path}...")
    # YOLO will auto-download when we try to load a model name
    model = YOLO('yolov8x-pose.pt')
    # Move it to the right location
    import shutil
    shutil.move('yolov8x-pose.pt', model_path)
    print(f"✓ Model saved to {model_path}")
else:
    print(f"✓ Model already exists at {model_path}")

✓ Model saved to ../models/pretrained/yolov8x-pose.pt


### Check GPU Availability

In [6]:
import torch
from ultralytics import YOLO

# Check if CUDA (GPU) is available
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
else:
    print("⚠️ No GPU detected. Training will use CPU (much slower).")

PyTorch version: 2.9.1+cu128
CUDA available: False
⚠️ No GPU detected. Training will use CPU (much slower).


### Train Model on Court Keypoints

In [14]:
# Train YOLO pose model on court keypoints with GPU
# Note: batch=-1 enables automatic batch size detection
# device=0 forces GPU usage (device='cpu' for CPU)
!yolo task=pose mode=train \
    model=../models/pretrained/yolov8x-pose.pt \
    data=../data/court-keypoints/data.yaml \
    epochs=100 \
    imgsz=640 \
    plots=True \
    batch=-1

New https://pypi.org/project/ultralytics/8.4.14 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.7 🚀 Python-3.12.3 torch-2.9.1+cu128 CPU (12th Gen Intel Core i9-12900H)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/court-keypoints/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../models/pretrained/yolov8x-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=0.

## Running Baseline YOLO on video clip

In [7]:
from ultralytics import YOLO

model = YOLO("/home/nlo/school/capstone/models/pretrained/yolov5l6u.pt")

results = model.predict("/home/nlo/school/capstone/input_videos/video_1.mp4",
                        save=True, 
                        show=True,
                        project="/home/nlo/school/capstone/runs",
                        name="yolov5l6u_video1")
#, conf=0.25, device=0)
print(results)
print("=============")

for box in results[0].boxes:
    print(box)


WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/117) /home/nlo/school/capstone/input_videos/video_1.mp4: 768x1280 19 persons, 1 sports ball, 1874.1ms
video 1/1 (frame 2/117) /home/nlo/school/capstone/input_videos/video_1.mp4: 768x1280 20 persons, 1 sports ball, 1398.0ms
video 1/1 (frame 3/117) /home/nlo/school/capstone/input_videos/video_1.mp4: 768x1280 19 persons, 1 sports ball, 1387.2ms
video 1/1 (frame 4/117) /home/nlo/school/capstone/input_videos/video_1.mp4: 768x1280 21 persons

## Running Finetuned YOLO on video clip

In [1]:
from ultralytics import YOLO

model = YOLO("/home/nlo/school/capstone/models/court-keypoints.pt")

results = model.predict("/home/nlo/school/capstone/input_videos/video_1.mp4",
                        save=True, 
                        show=True,
                        project="/home/nlo/school/capstone/runs",
                        name="court-keypoints_video1")
#, conf=0.25, device=0)
print(results)
print("=============")

for box in results[0].boxes:
    print(box)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/117) /home/nlo/school/capstone/input_videos/video_1.mp4: 288x512 1 basketball, 2070.9ms
video 1/1 (frame 2/117) /home/nlo/school/capstone/input_videos/video_1.mp4: 288x512 1 basketball, 773.4ms
video 1/1 (frame 3/117) /home/nlo/school/capstone/input_videos/video_1.mp4: 288x512 1 basketball, 1402.2ms
video 1/1 (frame 4/11

In [4]:
import os
os.environ['IMAGEIO_FFMPEG_EXE'] = 'auto'  # Use imageio-ffmpeg

import mediapy as mp

video_path = "/home/nlo/school/capstone/runs/court-keypoints_video1/video_1.avi" 
# Read the video
video = mp.read_video(video_path)
# Display the video
mp.show_video(video, width=500)

RuntimeError: Program 'ffmpeg' is not found; perhaps install ffmpeg using 'apt install ffmpeg'.